In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import time
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

def calculate_ma(series, window=30):
    """Calculate moving average for a series"""
    return pd.to_numeric(series, errors='coerce').rolling(window=window).mean()

def format_large_number(value):
    """
    Format large numbers to 3 digits with proper unit
    Example: 1234567890 -> 1.23B
    """
    abs_value = abs(float(value))
    if abs_value >= 1e12:
        return f"{value/1e12:.3f}T"
    elif abs_value >= 1e9:
        return f"{value/1e9:.3f}B"
    elif abs_value >= 1e6:
        return f"{value/1e6:.3f}M"
    elif abs_value >= 1e3:
        return f"{value/1e3:.3f}K"
    else:
        return f"{value:.3f}"

def get_scale_and_divisor(max_value):
    """
    Get the appropriate scale divisor and unit suffix for y-axis
    """
    abs_value = abs(float(max_value))
    if abs_value >= 1e12:
        return 1e12, "T"
    elif abs_value >= 1e9:
        return 1e9, "B"
    elif abs_value >= 1e6:
        return 1e6, "M"
    else:
        return 1e3, "K"

def get_and_plot_market_caps(ma_window=30):
    """
    Fetch and plot cryptocurrency market caps with configurable options
    
    Parameters:
    - ma_window: int, window size for moving average calculation (default 30 days)
    """
    # Read the CSV file
    df = pd.read_csv('coinmarketcap_20210103_top_125.csv')
    
    # Get top 10 assets to exclude
    top_10_assets = df.head(10)['Name'].tolist()
    print("Excluding these top 10 assets:", top_10_assets)
    
    # Create symbol mapping dictionary (excluding top 10)
    symbol_mapping = {
        row['Name']: f"{row['Symbol']}-USD"
        for _, row in df.iterrows()
        if row['Name'] not in top_10_assets
    }
    
    # Dictionary to store market caps for each crypto
    market_caps_dict = {}
    
    # Define the date range
    start_date = '2019-01-01'
    end_date = datetime.now().strftime('%Y-%m-%d')
    
    # Loop through each cryptocurrency
    for name, symbol in symbol_mapping.items():
        try:
            print(f"Fetching data for {name} ({symbol})...")
            
            # Fetch data from Yahoo Finance
            ticker = yf.Ticker(symbol)
            hist = ticker.history(start=start_date, end=end_date)
            
            if not hist.empty:
                # Calculate market cap (Close Price * Volume)
                market_caps_dict[name] = pd.to_numeric(hist['Close'] * hist['Volume'], errors='coerce')
            
            # Add a small delay to avoid hitting rate limits
            time.sleep(1)
            
        except Exception as e:
            print(f"Error fetching data for {name} ({symbol}): {str(e)}")
            continue
    
    # Also get Bitcoin data for dominance calculation
    try:
        print("Fetching Bitcoin data...")
        btc_ticker = yf.Ticker("BTC-USD")
        btc_hist = btc_ticker.history(start=start_date, end=end_date)
        if not btc_hist.empty:
            market_caps_dict['Bitcoin'] = pd.to_numeric(btc_hist['Close'] * btc_hist['Volume'], errors='coerce')
    except Exception as e:
        print(f"Error fetching Bitcoin data: {str(e)}")
        return
    
    # Combine all market caps into a single DataFrame
    if market_caps_dict:
        all_market_caps = pd.concat(market_caps_dict, axis=1)
        df_filled = all_market_caps.apply(pd.to_numeric, errors='coerce').ffill()
        
        # Calculate market metrics (excluding Bitcoin for others)
        others_market_cap = df_filled.drop('Bitcoin', axis=1).sum(axis=1)
        btc_dominance = (df_filled['Bitcoin'] / (df_filled['Bitcoin'] + others_market_cap)) * 100
        
        # Calculate moving averages
        others_ma = calculate_ma(others_market_cap, ma_window)
        dominance_ma = calculate_ma(btc_dominance, ma_window)
        
        # Get scale for formatting
        scale, unit_suffix = get_scale_and_divisor(others_market_cap.max())
        
        # Create figure with secondary y-axis
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        # Add market cap trace
        fig.add_trace(
            go.Scatter(
                x=others_market_cap.index,
                y=others_market_cap.astype(float) / scale,
                name="Total Market Cap (excl. Top 10)",
                mode='lines',
                line=dict(color='#000000', width=1),
                hovertemplate=f"Total Market Cap: $%{{y:.3f}}{unit_suffix}<extra></extra>"
            ),
            secondary_y=False
        )

        # Add MA for market cap
        fig.add_trace(
            go.Scatter(
                x=others_ma.index,
                y=others_ma.astype(float) / scale,
                name=f"{ma_window}D MA - Market Cap",
                mode='lines',
                line=dict(color='#000000', width=2, dash='dash'),
                hovertemplate=f"{ma_window}D MA: $%{{y:.3f}}{unit_suffix}<extra></extra>"
            ),
            secondary_y=False
        )

        # Add BTC dominance trace
        fig.add_trace(
            go.Scatter(
                x=btc_dominance.index,
                y=btc_dominance.astype(float),
                name="BTC Dominance",
                mode='lines',
                line=dict(color='#666666', width=1),
                hovertemplate="BTC Dominance: %{y:.1f}%<extra></extra>"
            ),
            secondary_y=True
        )

        # Add MA for dominance
        fig.add_trace(
            go.Scatter(
                x=dominance_ma.index,
                y=dominance_ma.astype(float),
                name=f"{ma_window}D MA - Dominance",
                mode='lines',
                line=dict(color='#666666', width=2, dash='dash'),
                hovertemplate=f"{ma_window}D MA: %{{y:.1f}}%<extra></extra>"
            ),
            secondary_y=True
        )

        # Update layout
        fig.update_layout(
            title=dict(
                text=f'Cryptocurrency Market Cap (Excluding Top 10) vs Bitcoin Dominance',
                font=dict(
                    color='#000000',
                    size=29,
                    weight='bold'
                ),
                x=0.055,
                y=0.94,
                xanchor='left',
                yanchor='top'
            ),
            width=1350,
            height=750,
            showlegend=True,
            legend=dict(
                orientation="h",
                yanchor="top",
                y=0.98,
                xanchor="left",
                x=0.02,
                font=dict(
                    color='#000000',
                    size=17
                ),
                bgcolor='rgba(255, 255, 255, 1)'
            ),
            plot_bgcolor='white',
            paper_bgcolor='white'
        )

        # Convert index to datetime if it's not already
        if not isinstance(others_market_cap.index, pd.DatetimeIndex):
            others_market_cap.index = pd.to_datetime(others_market_cap.index)

        # Update x-axis to show every 3 months
        date_range = pd.date_range(
            start=others_market_cap.index.min(), 
            end=others_market_cap.index.max(), 
            freq='3ME'
        )
        
        # Create two-line date labels
        date_labels = [f"{d.strftime('%b')}<br>{d.strftime('%Y')}" for d in date_range]
        
        fig.update_xaxes(
            ticktext=date_labels,
            tickvals=date_range,
            tickangle=0,
            showgrid=False,
            ticks='outside',
            ticklen=8,
            tickwidth=1,
            tickfont=dict(
                color='#000000',
                size=17
            ),
            linecolor='#000000',
            linewidth=1,
            mirror=True
        )

        # Calculate y-axis ranges and tick values
        y_max_market = np.ceil(others_market_cap.max() / scale)
        y_tick_vals_market = np.linspace(0, y_max_market, 6)  # Reduced number of ticks
        
        y_max_dom = np.ceil(btc_dominance.max() / 10) * 10
        y_tick_vals_dom = np.linspace(0, y_max_dom, 6)  # Reduced number of ticks

        # Update primary y-axis (Market Cap)
        fig.update_yaxes(
            title=None,
            showgrid=False,
            tickfont=dict(
                color='#000000',
                size=17
            ),
            ticktext=[f"${val:.3f}{unit_suffix}" for val in y_tick_vals_market],
            tickvals=y_tick_vals_market,
            ticks='outside',
            ticklen=8,
            tickwidth=1,
            range=[0, y_max_market],
            secondary_y=False,
            linecolor='#000000',
            linewidth=1,
            mirror=True
        )

        # Update secondary y-axis (Dominance)
        fig.update_yaxes(
            title=None,
            showgrid=False,
            tickfont=dict(
                color='#000000',
                size=17
            ),
            ticksuffix="%",
            ticks='outside',
            ticklen=8,
            tickwidth=1,
            range=[0, y_max_dom],
            secondary_y=True,
            linecolor='#000000',
            linewidth=1,
            mirror=True,
            tickformat=".1f"
        )

        # Save outputs
        fig.write_html("crypto_dominance.html")
        pd.DataFrame({
            'Total Market Cap (excl. Top 10)': others_market_cap.apply(format_large_number),
            'BTC Dominance (%)': btc_dominance,
            f'Market Cap {ma_window}D MA': others_ma.apply(format_large_number),
            f'Dominance {ma_window}D MA': dominance_ma
        }).to_csv('dominance_analysis.csv')
        
        print("Analysis completed! Check 'crypto_dominance_2021.html' and 'dominance_analysis_2021.csv' for results.")
        
        # Show the plot
        fig.show()
    else:
        print("No data was collected")

if __name__ == "__main__":
    get_and_plot_market_caps(ma_window=90)  # 30-day MA

$USDC-USD: possibly delisted; no price data found  (1d 2019-01-01 -> 2024-11-15)


Excluding these top 10 assets: ['Bitcoin', 'Ethereum', 'Tether', 'Litecoin', 'XRP', 'Polkadot', 'Bitcoin Cash', 'Cardano', 'Binance Coin', 'Chainlink']
Fetching data for USD Coin (USDC-USD)...
Fetching data for Wrapped Bitcoin (WBTC-USD)...
Fetching data for Bitcoin SV (BSV-USD)...
Fetching data for Stellar (XLM-USD)...
Fetching data for EOS (EOS-USD)...
Fetching data for Monero (XMR-USD)...
Fetching data for THETA (THETA-USD)...
Fetching data for TRON (TRX-USD)...
Fetching data for NEM (XEM-USD)...
Fetching data for VeChain (VET-USD)...
Fetching data for Tezos (XTZ-USD)...
Fetching data for Celsius (CEL-USD)...
Fetching data for Uniswap (UNI-USD)...
Fetching data for UNUS SED LEO (LEO-USD)...
Fetching data for Crypto.com Coin (CRO-USD)...
Fetching data for Dogecoin (DOGE-USD)...
Fetching data for Cosmos (ATOM-USD)...
Fetching data for Dai (DAI-USD)...
Fetching data for Binance USD (BUSD-USD)...
Fetching data for Neo (NEO-USD)...
Fetching data for Aave (AAVE-USD)...
Fetching data for S

Failed to get ticker 'PAX-USD' reason: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
$PAX-USD: possibly delisted; no timezone found


Fetching data for Qtum (QTUM-USD)...


$QTUM-USD: possibly delisted; no price data found  (1d 2019-01-01 -> 2024-11-15)


Fetching data for SwissBorg (CHSB-USD)...


Failed to get ticker 'CHSB-USD' reason: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
$CHSB-USD: possibly delisted; no timezone found


Fetching data for THORChain (RUNE-USD)...
Fetching data for Nano (NANO-USD)...
Fetching data for Hedera Hashgraph (HBAR-USD)...
Fetching data for Siacoin (SC-USD)...
Fetching data for HUSD (HUSD-USD)...
Fetching data for Energy Web Token (EWT-USD)...
Fetching data for Augur (REP-USD)...
Fetching data for TerraUSD (UST-USD)...
Fetching data for ABBC Coin (ABBC-USD)...
Fetching data for Celo (CELO-USD)...
Fetching data for Kyber Network (KNC-USD)...
Fetching data for NXM (NXM-USD)...
Fetching data for Ocean Protocol (OCEAN-USD)...
Fetching data for Theta Fuel (TFUEL-USD)...
Fetching data for Lisk (LSK-USD)...
Fetching data for Bitcoin Gold (BTG-USD)...
Fetching data for Verge (XVG-USD)...
Fetching data for HedgeTrade (HEDG-USD)...
Fetching data for Horizen (ZEN-USD)...
Fetching data for Band Protocol (BAND-USD)...
Fetching data for MaidSafeCoin (MAID-USD)...
Fetching data for Bancor (BNT-USD)...
Fetching data for Quant (QNT-USD)...
Fetching data for Holo (HOT-USD)...
Fetching data for ZB